# Домашнее задание №13.  Модель BERT и GPT

In [25]:
!pip install datasets
!pip install transformers
!pip install torchmetrics
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import pandas as pd

## Задание

1. Взять датасет
    https://huggingface.co/datasets/merionum/ru_paraphraser
    решить задачу парафраза

2.    (дополнительно необязательная задача)на выбор взять
    https://huggingface.co/datasets/sberquad
    https://huggingface.co/datasets/blinoff/medical_qa_ru_data
    натренировать любую модель для вопросно ответной системы
    как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER


## План решения
[0. Загрузка и предобработка данных](#section_0)

[1. Решение задания 1](#section_1)

[2. Решение задания 2](#section_2)

## 0. Загрузка и предобработка данных <a id='section_0'></a>

**Загрузка данных**

In [27]:
from datasets import load_dataset

corpus_train = load_dataset('merionum/ru_paraphraser', data_files='train.jsonl')
corpus_test = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Using custom data configuration merionum--ru_paraphraser-4a63244a00d42f51
Reusing dataset json (/root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4a63244a00d42f51/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration merionum--ru_paraphraser-4ab1b249bc6d5bf7
Reusing dataset json (/root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4ab1b249bc6d5bf7/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
corpus_train, corpus_test

(DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
}),
 DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
}))

In [5]:
df_train = pd.DataFrame(corpus_train['train'])
df_train['text'] =  list(zip(df_train['text_1'], df_train['text_2']))
df_train = df_train[:2000]
df_train.head()

,id,id_1,id_2,text_1,text_2,class,text
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,(Полицейским разрешат стрелять на поражение по...
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,(Право полицейских на проникновение в жилище р...
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,(Президент Египта ввел чрезвычайное положение ...
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,(Вернувшихся из Сирии россиян волнует вопрос т...
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,(В Москву из Сирии вернулись 2 самолета МЧС с ...


In [6]:
#Классы: 1 -- точные перефразировки, 0 -- почти парафразы, -1 -- непарафразы
df_train['class'].value_counts()

0     745
1     642
-1    613
Name: class, dtype: int64

In [7]:
df_test = pd.DataFrame(corpus_test['train'])
df_test['text'] =  list(zip(df_test['text_1'], df_test['text_2']))
df_test = df_test[:500]
df_test.head()

,id,id_1,id_2,text_1,text_2,class,text
0,25349,34420,34421,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,-1,"(Цены на нефть восстанавливаются, Парламент Сл..."
1,25360,34438,34439,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,-1,"(""Гоголь-центр"" покажет видеозапись скандально..."
2,25382,34475,34476,Агент: РФС вновь задерживает зарплату Фабио Ка...,СМИ: Агент Фабио Капелло грозится подать в суд...,-1,(Агент: РФС вновь задерживает зарплату Фабио К...
3,25423,34494,34519,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,-1,(День Победы в Москве обещает выдаться облачны...
4,25430,34531,34532,Посол РФ в США: Россия будет бороться с попытк...,Правительство запланировало заработать на лоте...,-1,(Посол РФ в США: Россия будет бороться с попыт...


In [8]:
#Классы: 1 -- точные перефразировки, 0 -- почти парафразы, -1 -- непарафразы
df_test['class'].value_counts()

-1    234
0     151
1     115
Name: class, dtype: int64

In [33]:
#LabelEncoder
y_train = df_train['class'].values
y_test = df_test['class'].values

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

label_list = le.classes_  #array(['-1', '0', '1'], dtype=object)
label_list

y_train = le.transform(y_train) #array([1, 1, 1, ..., 0, 0, 0])
y_test = le.transform(y_test)
#le.inverse_transform(y_train) #array(['0', '0', '0', ..., '-1', '-1', '-1'], dtype=object)

**Предобработка данных**

In [10]:
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer

model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [11]:
#Создаем датасеты
class ParaphraseDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, classes):
        self._classes = classes

        self.tokenizer = tokenizer
        #для каждого text возвращает батч с полями:
               #'inputs_ids' -- тензор размера (B,1,max_len) из id токенов
               #'token_type_ids' -- тензор размера (B,1,max_len) из id типов токенов
               #'attention_mask' -- тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внима
        self._txts = txts
                
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._classes[index]

In [36]:
train_dataset = ParaphraseDataset(df_train['text'], y_train)
test_dataset = ParaphraseDataset(df_test['text'], y_test)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=4,
                          shuffle=False,
                          num_workers=0)

In [13]:
for txt, cls in train_loader:
    print(txt)
    print(txt[0])
    print(txt[1])
    print(len(txt[0]))
    print(cls)
    break

[('"Библиотечка "РГ": Путешествие в Европу на автомобиле.', 'МЧС Тувы: Найдено тело последнего из 6 подростков, погибших под лавиной.', 'В "болотном деле" появилась 57-летняя фигурантка.', 'Жена псковского губернатора задекларировала активы во Франции.'), ('Трейлер к новому «Стартреку» пересняли в Lego.', 'Под лавиной в Туве нашли тело последнего пропавшего подростка.', 'В «болотном деле» появилась новая фигурантка.', 'У жены псковского губернатора обнаружилась французская собственность.')]
('"Библиотечка "РГ": Путешествие в Европу на автомобиле.', 'МЧС Тувы: Найдено тело последнего из 6 подростков, погибших под лавиной.', 'В "болотном деле" появилась 57-летняя фигурантка.', 'Жена псковского губернатора задекларировала активы во Франции.')
('Трейлер к новому «Стартреку» пересняли в Lego.', 'Под лавиной в Туве нашли тело последнего пропавшего подростка.', 'В «болотном деле» появилась новая фигурантка.', 'У жены псковского губернатора обнаружилась французская собственность.')
4
tensor([0

## 1. Решение задания 1 <a id='section_1'></a>

 Используем предобученную модель cointegrated/rubert-base-cased-dp-paraphrase-detection (два класса: 0 --non-paraphrase, 1 -- paraphrase)

**Предобученная модель cointegrated/rubert-base-cased-dp-paraphrase-detection**

In [38]:
#загрузка модели
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [39]:
#пример токенизации
text1 = df_test['text'][0][0]
text2 = df_test['text'][0][1]
batch = tokenizer(text1, text2, return_tensors='pt')
batch

{'input_ids': tensor([[  101, 59713,  1469, 20264, 26631,  5052,   102, 28362, 27443, 57650,
         37777, 12678,  5576,  1758, 50938,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [40]:
#пример работы
def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt')
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1)
    return proba[0] # 0(non-paraphrase), 1(paraphrase)

text1 = df_test['text'][6][0]
text2 = df_test['text'][6][1]
proba = compare_texts(text1,text2)
cl = torch.argmax(proba)

print(text1)
print(text2)
print(proba)
print('predicted class: ', cl.numpy())
print('real class: ', le.inverse_transform([y_test[6]]))

Самолет вернулся в аэропорт Новосибирска из-за стука в салоне
Самолет вернулся в новосибирский аэропорт из-за таинственного стука
tensor([0.0231, 0.9769])
predicted class:  1
real class:  ['1']


**Изменение последнего слоя модели cointegrated/rubert-base-cased-dp-paraphrase-detection**

In [41]:
#изменяем количество выходов в последнем слое классификации
import torch
import torch.nn as nn

model.classifier = nn.Linear(in_features=768, out_features=3, bias=True)
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [18]:
import torchmetrics
test_f1 = torchmetrics.F1Score()

for test_input, test_class in test_loader:
    with torch.inference_mode():
        batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        preds = torch.argmax(proba,dim=1) #torch.Size([B])
        target = test_class #torch.Size([B])
        
        test_f1(preds, target)

print(f'test f1_score: {test_f1.compute().item():.3f}')

test f1_score: 0.250


**Дообучение последнего слоя предобученной модели cointegrated/rubert-base-cased-dp-paraphrase-detection**

In [43]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [44]:
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.classifier.parameters()]))

Parameters full train: 177855747
Parameters transfer learning: 2307


In [45]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
# обучаем последний слой классификации
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)  

In [22]:
#дообучение модели и подсчет метрик на тренировочном и валидационном датасетах
import torch
train_f1 = torchmetrics.F1Score()
test_f1 = torchmetrics.F1Score()

epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss_train = 0.0
    for train_input, train_class in train_loader:
        batch = tokenizer(train_input[0], train_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        target = train_class  # torch.Size([B])
        
        #loss
        batch_loss = criterion(proba, target)
        total_loss_train += batch_loss.item()

        #f1
        preds = torch.argmax(proba,dim=1) #torch.Size([B]))
        train_f1(preds, target)
        
        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_test = 0.0
    for test_input, test_class in test_loader:
        batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        target = test_class  # torch.Size([B])
        
        #loss
        batch_loss = criterion(proba, target)
        total_loss_test += batch_loss.item()
       
       #f1
        preds = torch.argmax(proba,dim=1) #torch.Size([B]))
        test_f1(preds, target)
            
    print(
        f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train f1: {train_f1.compute().item(): .3f} \
        | Test Loss: {total_loss_test / len(test_dataset): .3f} \
        | Test f1: {test_f1.compute().item(): .3f}')
    train_f1.reset()
    test_f1.reset()

Epochs: 1 | Train Loss:  0.219         | Train f1:  0.641         | Test Loss:  0.216         | Test f1:  0.680
Epochs: 2 | Train Loss:  0.214         | Train f1:  0.672         | Test Loss:  0.213         | Test f1:  0.676
Epochs: 3 | Train Loss:  0.210         | Train f1:  0.683         | Test Loss:  0.214         | Test f1:  0.674
Epochs: 4 | Train Loss:  0.212         | Train f1:  0.689         | Test Loss:  0.213         | Test f1:  0.672
Epochs: 5 | Train Loss:  0.211         | Train f1:  0.686         | Test Loss:  0.213         | Test f1:  0.700
Epochs: 6 | Train Loss:  0.208         | Train f1:  0.707         | Test Loss:  0.215         | Test f1:  0.676
Epochs: 7 | Train Loss:  0.208         | Train f1:  0.704         | Test Loss:  0.214         | Test f1:  0.678
Epochs: 8 | Train Loss:  0.207         | Train f1:  0.707         | Test Loss:  0.216         | Test f1:  0.676
Epochs: 9 | Train Loss:  0.210         | Train f1:  0.692         | Test Loss:  0.215         | Test f1:

In [23]:
#метрика дообученной модели на валидационном датасете
test_f1 = torchmetrics.F1Score()
model.eval()

for test_input, test_class in test_loader:
    batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
    proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
    preds = torch.argmax(proba,dim=1) #torch.Size([B])
    target = test_class #torch.Size([B])
        
    test_f1(preds, target)

print(f'test f1_score: {test_f1.compute().item():.3f}')

test f1_score: 0.676


**Вывод:** дообучение последнего (классификационного) слоя модели привело к улучшению метрики.

**Замечания:** из-за слишком ресурсозатратных вычислений пришлось ограничить размеры тренировочного и тестового датасетов и производить дообучение на небольшом количестве эпох.

## 2. Решение задания 2 <a id='section_2'></a>

**Загрузка и просмотр данных**

In [130]:
import pandas as pd

#данные скачаны с https://huggingface.co/datasets/blinoff/medical_qa_ru_data
dataset = pd.read_csv('medical_qa_ru_data.csv')
dataset.head()

,date,categ,theme,desc,ans,spec10
0,"8 Октября 2017, 11:55",Оториноларингология,Применение Ларипронта.,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...,Отоларинголог
1,"20 Февраля 2019, 13:24",Акушерство,Беременность,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего...",NaN
2,"17 Марта 2015, 18:31",Другое,гинекология,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч,NaN
3,"13 Января 2019, 19:38",Терапия,Занятия спорта после сдачи крови,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч...",Терапевт
4,"28 Ноября 2017, 21:58",Другое,Таблетки,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...,NaN


In [131]:
df = dataset[['desc', 'ans']].rename(columns = {'desc': 'Question', 'ans': 'Answer'})
df

,Question,Answer
0,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...
1,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего..."
2,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч
3,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч..."
4,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...
...,...,...
190330,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю..."
190331,NaN,"В ""консультанте"""
190332,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ..."
190333,NaN,Такие препараты принимаются постоянно;\nЗдравс...


In [132]:
#удаляем NAN
full_data = df.dropna().reset_index()
full_data

,index,Question,Answer
0,0,"Ларипронт 20 талеток,через каждые 2-3 часа.Оче...",Что вы им лечите? Длительность приема Ларипрон...
1,1,"Здравствуйте, я на 7-8 неделе беременности. С ...","Здравствуйте, это может быть признаком раннего..."
2,2,Здравствуйте месячные должны придти 23 марта в...,Выполните исследование хгч
3,3,"Завтра иду с утра сдавать кровь ТТГ, Т4СВ, Кал...","Можно.;\nЗдравствуйте , да, попейте сладкого ч..."
4,4,Мне прописали пить Аллохол. Врач написала пить...,Препарат принимается после еды. Уточните это ...
...,...,...,...
171332,190328,"Не курю и не пью. Веду здоровый образ жизни, м...",В первую очередь вам необходимо обследовать пе...
171333,190329,"Здравствуйте, очень прошу консультации. В лево...",Начинать следует с терапевтических методов. Ес...
171334,190330,"Здравствуйте! В глаз отлетел кусок пластмассы,...","Ромашка есть дома, альбуцид;\nМожно промыть лю..."
171335,190332,"Здравствуйте, три дня назад порезала палец бу...","Здравствуйте , пока обработать Хлоргексидином ..."


**Предобработка данных**

In [136]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [133]:
#Функция преобразования текста (лемматизация, удаление стоп-слов и пунктуации)
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [134]:
#пример преобразования ответа
preprocess_txt(full_data['Answer'].values[0])

['лечить',
 'длительность',
 'приём',
 'ларипронт',
 'индивидуальный',
 'устанавливаться',
 'лечить',
 'врач',
 'уточнить',
 'свой',
 'лечить',
 'врач']

In [135]:
#Процедура подготовки данных к обучению (создание списка токенов для каждого текста)
assert True

# Preprocess for models fitting
answer_tokens = []
c = 0

for answer in full_data['Answer']:
    answer_prep = preprocess_txt(answer)
    answer_tokens.append(answer_prep)
    c += 1
    if c > 100000:
        break

In [137]:
# подготовленные токены answer
answer_tokens = [i for i in answer_tokens if len(i) > 2]
answer_tokens[0]

['лечить',
 'длительность',
 'приём',
 'ларипронт',
 'индивидуальный',
 'устанавливаться',
 'лечить',
 'врач',
 'уточнить',
 'свой',
 'лечить',
 'врач']

**Построение модели Word2Vec**

In [138]:
# Функция поиска k ближайших ответов
def get_response(question, index, model, index_map, count_answer=3):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, count_answer)
    return [index_map[i] for i in answers]

In [139]:
#инициализация модели
modelW2V = Word2Vec(sentences=answer_tokens, vector_size=300, window=5, min_count=3, workers=8)

In [140]:
#поиск ближайших соседей
w2v_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for i in range(len(full_data)):
    n_w2v = 0
    index_map[counter] = full_data['Answer'][i]
    question = preprocess_txt(full_data['Question'][i])
        
    vector_w2v = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
          
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        
    w2v_index.add_item(counter, vector_w2v)
            
    counter += 1

w2v_index.build(10)

True

**Пример работы модели**

In [147]:
TEXT = "Эрозия шейки матки. Куда идти?"

In [148]:
candidate_answer = get_response(TEXT, w2v_index, modelW2V, index_map, count_answer=3)

In [151]:
candidate_answer[0]

'Каков ваш вопрос?;\nДля лечения обратитесь к гинекологу,эрозию шейки матки заочно невозможно вылечить.;\nСформулируйте вопрос.'